In [ ]:
import pandas as pd

df = pd.read_csv('data/TB_RECIPE_SEARCH-20231130.csv')
df = df.rename(columns={
    'CKG_MTRL_CN': 'ingredients',
    'CKG_IPDC': 'sdescription',
    'CKG_NM': 'recipe_name'
})

df01 = df[['ingredients', 'sdescription', 'recipe_name']] 
df01

,ingredients,sdescription,recipe_name
0,[재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개|...,맛있는 김말이에 쫄깃함을 더한 어묵 김말이예요-,어묵김말이
1,[재료] 두부 1/2모| 당근 1/2개| 고추 2개| 브로콜리 1/4개| 새우 4마...,꼬리가 너-무- 매력적인 두부새우전. 두부와 야채를 한번에!! 영양까지 만점인 두부...,두부새우전
2,[재료] 밥 1+1/2공기| 당근 1/4개| 치자단무지 1/2개| 신김치 1쪽| 무...,간단하게 만들어 보는 알이 톡톡톡 알밥♥ 다 먹고 누룽지까지 싹싹 긁어먹는게 최고죠...,알밥
3,[재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간,현미호두죽,현미호두죽
4,[재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T|...,오늘은 집에서 굴러다니고 쉽게 구할 수 있는 북어로 일품요리를 만들어 보았어요! 도...,북어갈비
...,...,...,...
184986,[재료] 올리브오일| 스파게티면 80g| 명란젓 1덩이| 생크림 100g| 마늘 1...,이탈리아 사람들은 크림 파스타를 안 먹지만 제가 먹고 싶어서 만들어본 명란젓 크림 ...,명란젓크림스파게티
184987,[재료] 올리브오일| 스파게티면 280~300g| 관찰레 120g| 홀토마토 1캔|...,이 파스타가 먹고 싶었는데 관찰레를 구하기가 쉽지 않아 베이컨이나 판체타로 해보았는...,아마트리치아나스파게티
184988,[재료] 오징어젓200g| 무말랭이15g| 청양고추2개| 홍고추1개| 오이고추1개|...,밥도둑 젓갈무침 맛나게 먹구선 짜게 먹게돼 물 많이 먹게되는 젓갈무침 좋아하지만 절...,오징어젓무침
184989,[재료] 달걀 6개| 우유 200ml| 물 100ml| 맛살 3개| 당근 1/3개|...,안녕하세요 네이버 푸드인플루언서 류이입니다~^^ 중1 장남이를 176cm 진격의 ...,계란찜


In [5]:
from sqlalchemy import create_engine, text
# SQLite 데이터베이스 연결
engine = create_engine('sqlite:///recipes.db')

# 데이터프레임을 데이터베이스에 저장
df01.to_sql('recipes', engine, index=False, if_exists='replace')

# 데이터 저장 확인
with engine.connect() as connection:
    result = connection.execute(text("SELECT * FROM recipes LIMIT 5"))
    for row in result:
        print(row)

('[재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개| 깻잎 6장| 튀김가루 1컵 | 올리브유 적당량| 간장 1T| 참기름 1T', '맛있는 김말이에 쫄깃함을 더한 어묵 김말이예요-', '어묵김말이')
('[재료] 두부 1/2모| 당근 1/2개| 고추 2개| 브로콜리 1/4개| 새우 4마리| 녹말가루| 계란 1개', '꼬리가 너-무- 매력적인 두부새우전. 두부와 야채를 한번에!! 영양까지 만점인 두부새우전. 모양도 이쁘고 맛까지 좋은 두부새우전!! 함께 만들어 보아요♥', '두부새우전')
('[재료] 밥 1+1/2공기| 당근 1/4개| 치자단무지 1/2개| 신김치 1쪽| 무순 약간| 날치알 6스푼| 김가루 약간| 후리가케(또는밥이랑같은류)| 참기름 약간| 통깨 약간| 계란 노른자 2알', '간단하게 만들어 보는 알이 톡톡톡 알밥♥ 다 먹고 누룽지까지 싹싹 긁어먹는게 최고죠~ 색깔도 너무 이쁘고 한 그릇만 있으면 반찬도 필요없는 알밥', '알밥')
('[재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간', '현미호두죽', '현미호두죽')
('[재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T| 다진파 1T| 다진마늘 1T| 참기름 1T| 깨소금 1T| 후춧가루 약간', '오늘은 집에서 굴러다니고 쉽게 구할 수 있는 북어로 일품요리를 만들어 보았어요! 도시락 반찬으로는 물론 초대요리에도 너무너무 좋은 요리랍니다~ 겉에 찹쌀가루를 묻혀서 맛도 고소해요~', '북어갈비')


In [6]:
import pandas as pd
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 레시피 데이터 예시
data = df01.to_dict(orient='list')

recipes_df = pd.DataFrame(data).dropna()
recipes_df


,ingredients,sdescription,recipe_name
0,[재료] 어묵 2개| 김밥용김 3장| 당면 1움큼| 양파 1/2개| 당근 1/2개|...,맛있는 김말이에 쫄깃함을 더한 어묵 김말이예요-,어묵김말이
1,[재료] 두부 1/2모| 당근 1/2개| 고추 2개| 브로콜리 1/4개| 새우 4마...,꼬리가 너-무- 매력적인 두부새우전. 두부와 야채를 한번에!! 영양까지 만점인 두부...,두부새우전
2,[재료] 밥 1+1/2공기| 당근 1/4개| 치자단무지 1/2개| 신김치 1쪽| 무...,간단하게 만들어 보는 알이 톡톡톡 알밥♥ 다 먹고 누룽지까지 싹싹 긁어먹는게 최고죠...,알밥
3,[재료] 현미 4컵| 찹쌀 2컵| 호두 50g| 물 1/2컵| 소금 약간,현미호두죽,현미호두죽
4,[재료] 북어포 1마리| 찹쌀가루 1C [양념] 간장 2T| 설탕 1T| 물 1T|...,오늘은 집에서 굴러다니고 쉽게 구할 수 있는 북어로 일품요리를 만들어 보았어요! 도...,북어갈비
...,...,...,...
184986,[재료] 올리브오일| 스파게티면 80g| 명란젓 1덩이| 생크림 100g| 마늘 1...,이탈리아 사람들은 크림 파스타를 안 먹지만 제가 먹고 싶어서 만들어본 명란젓 크림 ...,명란젓크림스파게티
184987,[재료] 올리브오일| 스파게티면 280~300g| 관찰레 120g| 홀토마토 1캔|...,이 파스타가 먹고 싶었는데 관찰레를 구하기가 쉽지 않아 베이컨이나 판체타로 해보았는...,아마트리치아나스파게티
184988,[재료] 오징어젓200g| 무말랭이15g| 청양고추2개| 홍고추1개| 오이고추1개|...,밥도둑 젓갈무침 맛나게 먹구선 짜게 먹게돼 물 많이 먹게되는 젓갈무침 좋아하지만 절...,오징어젓무침
184989,[재료] 달걀 6개| 우유 200ml| 물 100ml| 맛살 3개| 당근 1/3개|...,안녕하세요 네이버 푸드인플루언서 류이입니다~^^ 중1 장남이를 176cm 진격의 ...,계란찜


In [7]:

def recommend_recipes(user_ingredients, recipes_df):
    
    # TF-IDF 벡터화
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(recipes_df['ingredients'])

    # 사용자 입력 재료를 벡터화
    user_vector = tfidf_vectorizer.transform([user_ingredients])

    # 코사인 유사도 계산
    cosine_similarities = cosine_similarity(user_vector, tfidf_matrix).flatten()

    # 유사도에 따라 레시피 추천
    recommended_indices = cosine_similarities.argsort()[-10:][::-1]  # 상위 3개 추천
    recommended_recipes = recipes_df.iloc[recommended_indices]

    return recommended_recipes


In [8]:
# 사용자 입력 재료
okt = Okt()
user_input = input("재료를 입력하세요(마늘,대파,당근,밀가루): ")
user_ingredients_list = okt.nouns(user_input)
print(user_ingredients_list)

['대파', '마늘', '우유']


In [9]:
user_ingredients = ' '.join(user_ingredients_list)
recommended_recipes = recommend_recipes(user_ingredients, recipes_df)
recommended_recipes_dataframe = pd.DataFrame(recommended_recipes[['recipe_name', 'ingredients', 'sdescription']])

print(f"검색단어:{user_ingredients}")
print("추천 레시피:")
recommended_recipes_dataframe

검색단어:대파 마늘 우유
추천 레시피:


,recipe_name,ingredients,sdescription
14465,쉐이크,[재료] 마| 꿀| 우유,'마'를 이용한 쉐이크 건강음료 마쉐이크. 아주 간단하게 만들 수 있는 영양간식인데...
128402,마우유,[재료] 마| 우유| 꿀,초스피드 마우유 만들기
13213,흙마늘,[재료] 마늘,흙마늘은 달작지근한 맛이나서 남녀노소누구나 편하고 쉽게 섭취할 수 있어요
28517,흑마늘,[재료] 마늘,고혈압 당뇨 노화 신경통 등 각종 성인병의 개선 및 치유에 큰 도움이 되는 흑...
48712,마늘손질,[재료] 마늘,기본재료중에 기본! 갈릭~ 마늘도 손다치지 않게 손질해봐요! 세상의 모든 레시피 만...
179511,마늘보관법,[재료] 마늘,마늘이 시간이 지날수록 상하기도하고 싹이 날려고 하네요. 이럴땐 까서 찧어서 냉동해...
94627,흑마늘,[재료] 마늘,노화방지 피로회복 면역력강화에 좋은 흑마늘 입니다
116850,마늘,[재료] 마늘,아릿아릿한 마늘 선별&손질&보관
48028,흑마늘,[재료] 마늘,이번 명절에 시골에서 토종마늘을 많이 주셔서 흑마늘을 만들었어요 오쿠로 하면 더 ...
92968,마늘꿀절임,[재료] 마늘| 꿀,따뜻한 성질을 가진 마늘과 꿀이 만나면 수족냉증에 효과적이고 몸이 찬 분들이 섭취하...
